# Using DFTpy mixer

In [1]:
import _qepy
import qepy
import importlib

In [2]:
import numpy as np
from qepy.driver import Driver
from qepy.io import QEInput

In [3]:
from ase.io.trajectory import Trajectory
from ase.lattice.hexagonal import Graphene
from ase import Atoms
import matplotlib.pyplot as plt

In [4]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'"
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'nat': 1,
        'ntyp': 1,
        'occupations': "'smearing'"
    },
    '&electrons': {
        'conv_thr' : 1e-8
    },
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'atomic_species': ['Al  26.98 Al_ONCV_PBE-1.2.upf'],
    'k_points automatic': ['4 4 4 0 0 0'],
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
}

In [5]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out')

In [6]:
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break
driver.calc_energy()

Iter:  0  - Conv:  0.07423450252710213
Iter:  1  - Conv:  0.0014686764583955348
Iter:  2  - Conv:  3.4654773793272354e-05
Iter:  3  - Conv:  6.680306265302417e-08
Iter:  4  - Conv:  2.5412863642447476e-09


-137.91449178319718

In [7]:
from dftpy.functional import Hartree

In [8]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.1.out')

In [9]:
rho = driver.get_density().copy()
coef = 0.7
for i in range(20):
    driver.diagonalize()
    #
    rho_new = driver.get_density().copy()
    drho = driver.data2field(rho_new-rho)
    error = Hartree.compute(drho).energy
    nc = np.abs(drho).sum()*driver.get_volume()/drho.size
    print ('Iter: ',i,' - Conv: ', error, 'dN:', nc)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    rho = (1-coef)*rho + coef * rho_new
    driver.set_density(rho)
driver.calc_energy()

Iter:  0  - Conv:  0.03712146707277951 dN: 0.6391793421661768
Iter:  1  - Conv:  0.0007346997237828299 dN: 0.15157146937277835
Iter:  2  - Conv:  3.914394446485253e-05 dN: 0.04556447497426659
Iter:  3  - Conv:  3.3366355611395276e-06 dN: 0.014013405977105845
Iter:  4  - Conv:  3.240568487976684e-07 dN: 0.004422327824047367
Iter:  5  - Conv:  3.072301214068241e-08 dN: 0.0013636509938231918
Iter:  6  - Conv:  2.813491299355543e-09 dN: 0.0004195906803282026


-137.914491783489

In [10]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.2.out')

In [11]:
rho_new = driver.get_density().copy()
coef = 0.7
v_h = driver.get_hartree_potential()
v_xc = driver.get_exchange_correlation_potential()
v_hxc = v_h + v_xc
for i in range(10):
    driver.diagonalize()
    #
    rho_new, rho = driver.get_density().copy(), rho_new
    drho = driver.data2field(rho_new-rho)
    error = Hartree.compute(drho).energy
    nc = np.abs(drho).sum()*driver.get_volume()/drho.size
    print ('Iter: ',i,' - Conv: ', error, 'dN:', nc)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    v_h = driver.get_hartree_potential()
    v_xc = driver.get_exchange_correlation_potential()
    v_hxc_new = v_h + v_xc
    v_hxc = (1-coef)*v_hxc + coef*v_hxc_new
    driver.set_external_potential(v_hxc, exttype=('hartree', 'xc'))
driver.get_energy()

Iter:  0  - Conv:  0.037130390200344775 dN: 0.6392307603451793
Iter:  1  - Conv:  0.0013217442607765945 dN: 0.11560534604791975
Iter:  2  - Conv:  1.3612362253022785e-05 dN: 0.012083919530383689
Iter:  3  - Conv:  3.5529364273468253e-07 dN: 0.002094837919952711
Iter:  4  - Conv:  8.690632525165175e-09 dN: 0.00045731318739506696


-137.91449178319868

In [12]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.3.out')

In [13]:
from dftpy.mixer import Mixer
driver.mixer = Mixer(scheme='pulay')

In [14]:
rho = driver.data2field(driver.get_density().copy())
for i in range(10):
    driver.diagonalize()
    #
    rho_new = driver.data2field(driver.get_density().copy())
    drho = rho_new - rho
    error = Hartree.compute(drho).energy
    nc = np.abs(drho).integral()
    print ('Iter: ',i,' - Conv: ', error, 'dN:', nc)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    rho = driver.mixer(rho, rho_new, coef=0.7)
    density = driver.field2data(rho)
    driver.set_density(density)
driver.calc_energy()

Iter:  0  - Conv:  0.03711755355705845 dN: 0.6391999808393399
Iter:  1  - Conv:  0.0007355225168336646 dN: 0.15165749731110584
Iter:  2  - Conv:  1.3088974716907787e-05 dN: 0.027156939318301748
Iter:  3  - Conv:  5.904945025604013e-08 dN: 0.0011748998563695147
Iter:  4  - Conv:  3.517204811622727e-10 dN: 0.00013735679623787827


-137.91449178192417

In [15]:
ions = driver.get_dftpy_ions()

In [16]:
rho.write('tmp.xsf', ions=ions)